## Classification of the MNIST data using CNNs

The MNIST Dataset has 28 x 28 black and white images of digits from 0 to 9. It is one of the most common datasets for starting up with deep learning. It comes in built in the keras package.

This notebook will walk you through the developing a classification model for the dataset using a <b>Convolutional Neural Network</b> with the help of the <b>Model API</b> from Keras.

### Importing required libraries

In [1]:
import os

os.environ['KERAS_BACKEND'] = 'tensorflow'

import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
from matplotlib import pyplot as plt

from keras.layers import Input, Flatten, Dense, Dropout, MaxPooling2D
#from keras.layers.advanced_activations import LeakyReLU
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.convolutional import Conv2D

from keras.activations import *
from keras.optimizers import *
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.models import Model, Sequential

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Loading the dataset

MNIST comes as a part of the keras datasets. It contains 60,000 training images while 10,000 test images

In [2]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Display one image?

In [3]:
X_train.shape, X_test.shape

((60000, 28, 28), (10000, 28, 28))

### Data Preprocessing

The labels refer to the expected classification output for a given image. They are converted from singular to one-hot encoded values from 0 to 9. 

That is, if a given image corresponds to 5, its encoding will be [0,0,0,0,0,1,0,0,0,0]

- Since we are using a convolutional network we do not need to flatten the array into 1D.
- rescale pixel values between 0 and 1\n
- Input type should be float\n
- There are 10 classes so in order to compute the cross entropy loss function we need to one-hot encoded vectors.

In [4]:
X_train = (X_train.astype(np.float32) - 127.5)/127.5
X_test = (X_test.astype(np.float32) - 127.5)/127.5

X_train = X_train[:,:,:,np.newaxis]
X_test = X_test[:,:,:,np.newaxis]

X_train.shape, X_test.shape

((60000, 28, 28, 1), (10000, 28, 28, 1))

In [5]:
num_classes = 10

# convert class vectors to binary class matrices
y_train = to_categorical(y_train, num_classes)
#y_test = to_categorical(y_test, num_classes)

### Creating a Keras Model

Construct a model using the sequential API with the following instructions:

- Inputs are normalized using BatchNormaliation followed by a Dropout layer with a rate of 0.3
- Then add a 2D convolutional layer with a kernel of 3x3 with 32 filters
- Activate the output using an Activation layer
- Output from the convolutional layer goes through a MaxPooling layer
- Add another 2D convolutional layer but ensure that the shape remains 'same' as previous layer having 64 filters
- Output from the convolutional layer goes through a MaxPooling layer
- Then Flatten the output and add a Dropout layer with a rate of 0.3
- Connect the output to a Dense layer containing 100 neurons followed by a BatchNormalization 
- Add 'relu' activation function
- and finally connect to the output layer with an softmax activation function

Print the model summary to see the network

In [6]:
In = Input(shape=(28,28,1))
conv = Conv2D(8, kernel_size=5, strides=1, padding="valid",activation='relu')
pool = MaxPooling2D(pool_size=2,strides=2, padding="same")
fc = Dense(num_classes, activation='softmax')

fmap1 = conv(In)
fmap2 = pool(fmap1)
dense = Flatten()(fmap2)
y = Dense(num_classes, activation='softmax')(dense)
# x = BatchNormalization()(x)

# x = Conv2D(64, kernel_size=3, strides=2, padding="same",activation='relu')(x)
# x = MaxPooling2D(pool_size=2,strides=2, padding="same")(x)
# x = BatchNormalization()(x)

# x = Flatten()(x)
# x = Dense(64, activation='relu')(x)
# y = Dense(num_classes, activation='softmax')(x)
                                                                                    

"""
model.add(layers.Dropout(0.3, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation = "relu")
model.add(layers.Dropout(0.2, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation = "relu"))
"""                                                                                

model = Model(inputs=In,outputs=y)                                                                                   
                                                                                   
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 8)         208       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 8)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1152)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                11530     
Total params: 11,738
Trainable params: 11,738
Non-trainable params: 0
_________________________________________________________________


The final model is compiled using an optimizer, a loss function and a metric for performance improvement. 
- The loss function is used to depict how far is the current model from the ideal answer
- The optimizer refers to the method that will be used to minimize the loss
- The metrics correspond to how we want to measure the performance of the network

In [7]:
model.compile(loss='categorical_crossentropy', optimizer=SGD(), metrics=['accuracy'])

### Training and testing

- Training is done using the function fit(). We train out network for 5 epochs.
- Testing is done using the predict() function. We can also use evaluate() but since we want to later generate a classificiation report, we will use the former


In [8]:
model.fit(X_train,y_train, epochs=5)

Epoch 1/5
60000/60000 [==============================] - 19s 323us/step - loss: 0.4035 - acc: 0.8852
Epoch 2/5
60000/60000 [==============================] - 18s 300us/step - loss: 0.1513 - acc: 0.9564
Epoch 3/5
60000/60000 [==============================] - 18s 305us/step - loss: 0.1167 - acc: 0.9663
Epoch 4/5
60000/60000 [==============================] - 19s 314us/step - loss: 0.0992 - acc: 0.9711
Epoch 5/5
60000/60000 [==============================] - 18s 305us/step - loss: 0.0881 - acc: 0.9745


In [9]:
y_check = model.predict(X_test)

y_pred = np.array([np.argmax(y_check[j]) for j in range(len(y_check))])
y_test

array([7, 2, 1, ..., 4, 5, 6], dtype=uint8)

In [10]:
print(confusion_matrix(y_test, y_pred))

[[ 975    0    1    0    0    0    0    2    2    0]
 [   0 1126    5    0    0    1    0    0    3    0]
 [   3    4 1008    2    3    0    1    5    6    0]
 [   1    0    4  982    0    5    0    6    8    4]
 [   1    1    3    0  970    0    1    2    1    3]
 [   4    1    0    7    0  872    2    1    4    1]
 [  10    5    1    1    3    3  927    1    7    0]
 [   1    1   15    3    0    0    0 1001    3    4]
 [   7    1    8    3    2    1    1    9  940    2]
 [   7    5    1    2    7    4    0   19    6  958]]


In [11]:
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.97      0.99      0.98       980
          1       0.98      0.99      0.99      1135
          2       0.96      0.98      0.97      1032
          3       0.98      0.97      0.98      1010
          4       0.98      0.99      0.99       982
          5       0.98      0.98      0.98       892
          6       0.99      0.97      0.98       958
          7       0.96      0.97      0.97      1028
          8       0.96      0.97      0.96       974
          9       0.99      0.95      0.97      1009

avg / total       0.98      0.98      0.98     10000



In [75]:
#conv.get_weights

In [76]:
x = []
for layer in model.layers: 
    x.append(layer.get_weights())
    #print(layer.get_weights())
    #print("\n\n\n\n")
    
wmap1 = np.array(x[1][0])
bmap1 = np.array(x[1][1])

wmap2 = np.array(x[4][0])
bmap2 = np.array(x[4][1])

#layer.get_config(), 

In [73]:
# bmap2 = np.array(x[4][1]).flatten()
# bmap2.shape
f1 = open("wmap1.txt","w")
f = wmap1.T.flatten('C')
for i in range(11520/5):
    try:
        x =  str(f[i])+", "+str(f[i+1])+", "+str(f[i+2])+", "+str(f[i+3])+", "+str(f[i+4])+", \n"
        f1.write(x)
    except:
        break
f1.close()

In [74]:
# wmap1 = np.array(x[1][0])
# wmap2 = np.array(x[4][0])
# #writer = wmap1.flatten()
# #wmap1
wmap1.T

array([[[[ 1.16742313e-01,  1.82521775e-01,  8.53309184e-02,
          -1.51690766e-01, -1.05779367e-02],
         [ 1.54524758e-01,  1.02859356e-01, -2.59427428e-01,
           1.28623629e-02,  2.15627402e-01],
         [ 2.29107872e-01, -7.71002099e-02, -3.05824250e-01,
           4.90354709e-02,  6.32800385e-02],
         [ 5.27418740e-02, -1.33307993e-01, -1.77181229e-01,
           3.08477223e-01, -7.94616640e-02],
         [-3.19887698e-02, -1.24499179e-01,  5.33855669e-02,
           2.42513716e-01,  9.36099365e-02]]],


       [[[ 5.55352792e-02,  5.54543138e-02, -5.09911552e-02,
          -8.44349787e-02, -1.10472500e-01],
         [ 7.55098881e-05, -7.25525692e-02, -1.53119892e-01,
          -1.56516992e-02, -2.17455208e-01],
         [-9.85072404e-02,  1.07371947e-02, -1.87309816e-01,
          -1.00623891e-01, -1.26578689e-01],
         [ 6.07124828e-02, -9.00263041e-02, -8.61675590e-02,
          -1.80530474e-01, -2.57794373e-02],
         [-1.52072012e-01, -1.05039939e-01

In [72]:
wmap = np.zeros((200,1))
dx = 0
for i in range(5):
    for j in range(5):
        for k in range(8):
            wmap[dx] = wmap1[i][j][0][k]
            dx += 1
wmap.T

array([[ 1.16742313e-01,  5.55352792e-02, -7.61901140e-02,
         5.39284408e-01, -2.09182113e-01, -1.78037643e-01,
         3.90237421e-01, -2.15333253e-01,  1.54524758e-01,
         7.55098881e-05, -1.38261288e-01,  3.63211483e-01,
        -2.87931636e-02, -1.39477476e-01,  6.52614057e-01,
        -3.22264999e-01,  2.29107872e-01, -9.85072404e-02,
        -1.35328501e-01,  2.28702769e-01, -2.62545258e-01,
        -1.42032087e-01,  6.27099693e-01, -2.08277598e-01,
         5.27418740e-02,  6.07124828e-02, -1.14080630e-01,
        -3.47269386e-01, -1.37009859e-01,  3.95289570e-01,
         6.24461353e-01, -1.60556898e-01, -3.19887698e-02,
        -1.52072012e-01,  1.43811330e-01, -4.33153123e-01,
        -1.65634707e-01,  3.77474278e-01,  5.36893547e-01,
        -2.01450378e-01,  1.82521775e-01,  5.54543138e-02,
        -1.26140311e-01,  5.00761449e-01, -2.15790182e-01,
        -8.37143883e-02,  2.80732781e-01, -3.14082891e-01,
         1.02859356e-01, -7.25525692e-02, -6.92386776e-0

In [60]:
#X_train[99].reshape((28,28,1))
#== wmap1.T.flatten(order='C')

In [107]:
#wmap1 = np.array(x[1][0])
writer = wmap1.T.flatten(order='C')
wmap1_base = 0x1FFFFC44
f=open("init.ini", "w")
for entry in writer:
    str1="_WFLOAT ("+str(wmap1_base)+","+str(entry)+") \n"
    f.write(str1)
    wmap1_base += 4
f.close()

# wmap = np.zeros((11520,1))
# dx = 0
# for i in range(1152):
#     for j in range(10):
#         #print i,j, dx
#         wmap[dx] = wmap2[i][j]
#         dx += 1
# wmap2 = wmap.flatten(order='C')

In [108]:
wmap2_base = 0x1FFFFF64
#wmap2 = np.array(x[4][0])
writer = wmap2.T.flatten(order='C')
f=open("init.ini", "a")
for entry in writer:
    str1="_WFLOAT ("+str(wmap2_base)+","+str(entry)+") \n"
    f.write(str1)
    wmap2_base += 4
f.close()

In [109]:
input_base = 0x1FFFF004
input_map = X_train[257]
writer = input_map.flatten(order='C')
f=open("init.ini", "a")
idx=0
for entry in writer:
    str1="_WFLOAT ("+str(input_base)+","+str(entry)+") \n"
    f.write(str1)
    #print str1
    input_base += 4
    idx+=1
f.close()
idx

784

In [110]:
x

[[], [array([[[[ 1.16742313e-01,  5.55352792e-02, -7.61901140e-02,
             5.39284408e-01, -2.09182113e-01, -1.78037643e-01,
             3.90237421e-01, -2.15333253e-01]],
  
          [[ 1.54524758e-01,  7.55098881e-05, -1.38261288e-01,
             3.63211483e-01, -2.87931636e-02, -1.39477476e-01,
             6.52614057e-01, -3.22264999e-01]],
  
          [[ 2.29107872e-01, -9.85072404e-02, -1.35328501e-01,
             2.28702769e-01, -2.62545258e-01, -1.42032087e-01,
             6.27099693e-01, -2.08277598e-01]],
  
          [[ 5.27418740e-02,  6.07124828e-02, -1.14080630e-01,
            -3.47269386e-01, -1.37009859e-01,  3.95289570e-01,
             6.24461353e-01, -1.60556898e-01]],
  
          [[-3.19887698e-02, -1.52072012e-01,  1.43811330e-01,
            -4.33153123e-01, -1.65634707e-01,  3.77474278e-01,
             5.36893547e-01, -2.01450378e-01]]],
  
  
         [[[ 1.82521775e-01,  5.54543138e-02, -1.26140311e-01,
             5.00761449e-01, -2.15790182e-01

In [111]:
bmap1 = np.array(x[1][1])
#print bmap1.shape
writer = bmap1.flatten(order='C')
bmap1_base = 0x2000B364 #020
f=open("init.ini", "a")
for entry in writer:
    str1="_WFLOAT ("+str(bmap1_base)+","+str(entry)+") \n"
    f.write(str1)
    bmap1_base += 4
f.close()

In [112]:
bmap2 = np.array(x[4][1])
#print bmap2.shape
writer = bmap2.flatten(order='C')
bmap2_base = 0x2000B384 #020
f=open("init.ini", "a")
for entry in writer:
    str1="_WFLOAT ("+str(bmap2_base)+","+str(entry)+") \n"
    f.write(str1)
    bmap2_base += 4
f.close()

In [90]:
bmap1

array([ 0.163432  , -0.1988819 , -0.08055043,  0.00747085, -0.04554225,
        0.00394376,  0.05191392,  0.19038282], dtype=float32)

In [102]:
mo = 257
np.argmax(y_train[mo])

4

In [103]:
model.predict(X_train[mo][np.newaxis,:,:,:])

array([[8.0121836e-06, 3.1961011e-07, 2.1982105e-06, 6.1066322e-07,
        9.9987161e-01, 2.5755538e-09, 2.9442068e-05, 1.8440036e-05,
        5.8084752e-05, 1.1305920e-05]], dtype=float32)

In [267]:
p = X_train[200].flatten('C')

f1 = open("image.txt","w")
for i in range(len(p)-1):
    x = str(p[i])+","
    f1.write(x)
x = str(p[i])
f1.write(x)
f1.close()

In [ ]:
len()

In [247]:
y_train[99]

array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [188]:
hex(0x1FF00000 + 128*1024*1024)

'0x27f00000'